In [2]:
PATH = os.path.join(os.getcwd() , 'D:\Projects\google_ocr\code\My Project-b659c3bd30fa.json')

In [1]:
try:
    from google.cloud import storage
    import google.cloud.storage
    import json
    import os
    import sys
except Exception as e:
    print("Error : {} ".format(e))

In [3]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=PATH


In [4]:
import base64
import os
import re
import csv
import io
import tempfile
import ghostscript
import locale
import glob
import time

from pydub import AudioSegment

from google.cloud import storage
from google.cloud import vision
#from google.cloud import texttospeech
# from google.cloud import automl_v1beta1 as automl
from google.protobuf import json_format

In [5]:


storage_client = storage.Client(PATH)

storage_client


!pip install --upgrade google-cloud-vision

import io
import os

# Imports the Google Cloud client library
from google.cloud import vision

# Instantiates a client
client = vision.ImageAnnotatorClient()

# The name of the image file to annotate
file_name = os.path.abspath(r'D:\Projects\google_ocr\code\Page_1.jpg')

# Loads the image into memory
with io.open(file_name, 'rb') as image_file:
    content = image_file.read()

image = vision.Image(content=content)

# Performs label detection on the image file
response = client.label_detection(image=image)
labels = response.label_annotations

print('Labels:')
for label in labels:
    print(label.description)

def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    print('Texts:')

    for text in texts:
        print('\n"{}"'.format(text.description))

        vertices = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in text.bounding_poly.vertices])

        print('bounds: {}'.format(','.join(vertices)))

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

text=detect_text('D:\Projects\google_ocr\code\Page_1.jpg')

In [6]:

bucket = storage_client.get_bucket('ocr_files_adil')

filename = [filename.name for filename in list(bucket.list_blobs(prefix='')) ]
filename



['2020_Resume1.0.pdf',
 'Resume1 output-1-to-1.json',
 '_Probability__and_Statistics___in___Data___Science___using___Python_Syllabus.pdf',
 'resume.pdf']

In [7]:
m = re.match("(.*).output-([0-9]+)-.*","Resume1 output-1-to-1.json")
pdf_id = m.group(1)
first_page = int(m.group(2))
#json_blob = os.path.join(os.getcwd() , 'D:\Projects\google_ocr\code\My Project-b659c3bd30fa.json')
json_blob="Resume1 output-1-to-1.json"

In [16]:
def build_feature_csv(json_blob, pdf_id, first_page):
    
    # parse json
    #json_string = json_blob.download_as_string()
    json_string = bucket.blob(blob_name =json_blob).download_as_string()
    json_response = json_format.Parse(json_string, vision.AnnotateFileResponse())

    # covert the json file to a bag of CSV lines
    csv = ""
    page_count = first_page
    for resp in json_response.responses:
        para_count = 0
        for page in resp.full_text_annotation.pages:

            # collect para features for the page
            page_features = []
            for block in page.blocks:
                if str(block.block_type) != "1":  # process only TEXT blocks
                    continue
                for para in block.paragraphs:
                    para_id = "{}-{:03}-{:03}".format(pdf_id, page_count, para_count)
                    f = extract_paragraph_feature(para_id, para)
                    page_features.append(f)
                    para_count += 1

            # output to csv
            for f in page_features:
                csv += '{},"{}",{},{:.6f},{:.6f},{:.6f},{:.6f},{:.6f},{:.6f},{:.6f},{}\n'.format(
                    f["para_id"],
                    f["text"],
                    f["chars"],
                    f["width"],
                    f["height"],
                    f["area"],
                    f["char_size"],
                    f["pos_x"],
                    f["pos_y"],
                    f["aspect"],
                    f["layout"],
                )

        page_count += 1
    return csv

In [ ]:
build_feature_csv(json_blob, pdf_id, first_page)

In [ ]:
json_string = bucket.blob(blob_name =json_blob).download_as_string()
with open(json_string,"wb") as read_file:
    data = json.load(read_file)
data

In [ ]:


blop = bucket.blob(blob_name ="Resume1 output-1-to-1.json").download_as_string()

In [ ]:
blop

In [10]:
def async_detect_document(gcs_source_uri, gcs_destination_uri):
    """OCR with PDF/TIFF as source files on GCS"""
    import json
    import re
    from google.cloud import vision
    from google.cloud import storage

    # Supported mime_types are: 'application/pdf' and 'image/tiff'
    mime_type = 'application/pdf'

    # How many pages should be grouped into each json output file.
    batch_size = 2

    client = vision.ImageAnnotatorClient()

    feature = vision.Feature(
        type_=vision.Feature.Type.DOCUMENT_TEXT_DETECTION)

    gcs_source = vision.GcsSource(uri=gcs_source_uri)
    input_config = vision.InputConfig(
        gcs_source=gcs_source, mime_type=mime_type)

    gcs_destination = vision.GcsDestination(uri=gcs_destination_uri)
    output_config = vision.OutputConfig(
        gcs_destination=gcs_destination, batch_size=batch_size)

    async_request = vision.AsyncAnnotateFileRequest(
        features=[feature], input_config=input_config,
        output_config=output_config)

    operation = client.async_batch_annotate_files(
        requests=[async_request])

    print('Waiting for the operation to finish.')
    operation.result(timeout=420)

    # Once the request has completed and the output has been
    # written to GCS, we can list all the output files.
    storage_client = storage.Client()

    match = re.match(r'gs://([^/]+)/(.+)', gcs_destination_uri)
    bucket_name = match.group(1)
    prefix = match.group(2)

    bucket = storage_client.get_bucket(bucket_name)

    # List objects with the given prefix.
    blob_list = list(bucket.list_blobs(prefix=prefix))
    print('Output files:')
    for blob in blob_list:
        print(blob.name)

    # Process the first output file from GCS.
    # Since we specified batch_size=2, the first response contains
    # the first two pages of the input file.
    output = blob_list[0]

    json_string = output.download_as_string()
    response = json.loads(json_string)
    m = re.match("(.*).output-([0-9]+)-.*","Resume1 output-1-to-1.json")
    pdf_id = m.group(1)
    first_page = int(m.group(2))
    json_blob="Resume1 output-1-to-1.json"
    
    # The actual response for the first page of the input file.
    first_page_response = response['responses'][0]
    annotation = first_page_response['fullTextAnnotation']
    # covert the json file to a bag of CSV lines
    csv = ""
    page_count = first_page
    for resp in response.responses:
        para_count = 0
        for page in resp.full_text_annotation.pages:

            # collect para features for the page
            page_features = []
            for block in page.blocks:
                if str(block.block_type) != "1":  # process only TEXT blocks
                    continue
                for para in block.paragraphs:
                    para_id = "{}-{:03}-{:03}".format(pdf_id, page_count, para_count)
                    f = extract_paragraph_feature(para_id, para)
                    page_features.append(f)
                    para_count += 1

            # output to csv
            for f in page_features:
                csv += '{},"{}",{},{:.6f},{:.6f},{:.6f},{:.6f},{:.6f},{:.6f},{:.6f},{}\n'.format(
                    f["para_id"],
                    f["text"],
                    f["chars"],
                    f["width"],
                    f["height"],
                    f["area"],
                    f["char_size"],
                    f["pos_x"],
                    f["pos_y"],
                    f["aspect"],
                    f["layout"],
                )

        page_count += 1
    return csv

    # Here we print the full text from the first page.
    # The response contains more information:
    # annotation/pages/blocks/paragraphs/words/symbols
    # including confidence scores and bounding boxes
    print('Full text:\n')
    print(annotation['text'])

In [11]:
async_detect_document('gs://ocr_files_adil/resume.pdf','gs://ocr_files_adil/Resume1 ')

Waiting for the operation to finish.
Output files:
Resume1 output-1-to-1.json


NameError: name 'json_response' is not defined

In [8]:
def build_feature_csv(json_blob, pdf_id, first_page):

    # parse json
    json_string = json_blob.download_as_string()
    json_response = json_format.Parse(json_string, vision.Enum.AnnotateFileResponse())

    # covert the json file to a bag of CSV lines
    csv = ""
    page_count = first_page
    for resp in json_response.responses:
        para_count = 0
        for page in resp.full_text_annotation.pages:

            # collect para features for the page
            page_features = []
            for block in page.blocks:
                if str(block.block_type) != "1":  # process only TEXT blocks
                    continue
                for para in block.paragraphs:
                    para_id = "{}-{:03}-{:03}".format(pdf_id, page_count, para_count)
                    f = extract_paragraph_feature(para_id, para)
                    page_features.append(f)
                    para_count += 1

            # output to csv
            for f in page_features:
                csv += '{},"{}",{},{:.6f},{:.6f},{:.6f},{:.6f},{:.6f},{:.6f},{:.6f},{}\n'.format(
                    f["para_id"],
                    f["text"],
                    f["chars"],
                    f["width"],
                    f["height"],
                    f["area"],
                    f["char_size"],
                    f["pos_x"],
                    f["pos_y"],
                    f["aspect"],
                    f["layout"],
                )

        page_count += 1
    return csv

In [ ]:
build_feature_csv('gs://ocr_files_adil/resume.pdf','gs://ocr_files_adil/Resume1 ',pdf_id, first_page)

In [ ]:
m = re.match("(.*).output-([0-9]+)-.*","Resume1 output-1-to-1.json")
pdf_id = m.group(1)
first_page = int(m.group(2))
#json_blob = os.path.join(os.getcwd() , 'D:\Projects\google_ocr\code\My Project-b659c3bd30fa.json')
json_blob="Resume1 output-1-to-1.json"

In [9]:
build_feature_csv(json_blob, pdf_id, first_page)

AttributeError: 'str' object has no attribute 'download_as_string'

In [ ]:
#Very Importent Code
from google.cloud import vision_v1


def sample_batch_annotate_files(
    storage_uri="gs://cloud-samples-data/vision/document_understanding/kafka.pdf",
):
    """Perform batch file annotation."""
    mime_type = "application/pdf"

    client = vision_v1.ImageAnnotatorClient()

    gcs_source = {"uri": storage_uri}
    input_config = {"gcs_source": gcs_source, "mime_type": mime_type}
    features = [{"type_": vision_v1.Feature.Type.DOCUMENT_TEXT_DETECTION}]

    # The service can process up to 5 pages per document file.
    # Here we specify the first, second, and last page of the document to be
    # processed.
    pages = [1, 2, -1]
    requests = [{"input_config": input_config, "features": features, "pages": pages}]

    response = client.batch_annotate_files(requests=requests)
    for image_response in response.responses[0].responses:
        print(u"Full text: {}".format(image_response.full_text_annotation.text))
        for page in image_response.full_text_annotation.pages:
            for block in page.blocks:
                print(u"\nBlock confidence: {}".format(block.confidence))
                for par in block.paragraphs:
                    print(u"\tParagraph confidence: {}".format(par.confidence))
                    for word in par.words:
                        print(u"\t\tWord confidence: {}".format(word.confidence))
                        for symbol in word.symbols:
                            print(
                                u"\t\t\tSymbol: {}, (confidence: {})".format(
                                    symbol.text, symbol.confidence
                                )
                            )
